<p>Вариант 1: Дан граф, описывающий возможность перемещений между локациями. Команда из нескольких героев находится в разных вершинах этого графа. В других вершинах графа находятся монстры. Каждый ход монстр с некоторой вероятностью решает покинуть текущую локацию и перейти в новую, связанную с текущей, либо остаться в текущей вершине. Вероятности перехода монстра в одну из локаций, связанных с текущей, равны. Герой за один ход может совершить лишь один переход в соседнюю локацию. Соберите всех героев в финишной локации.</p>

<h3>Мое прочтение задачи:</h3>
<p>-герои формально ходят после монстров</p>
<p>-но на самом деле они ходят одновременно, поэтому героям нет смылса пытаться предсказать, есть ли рядом монстр, потому что в тот же ход это уже может быть неактуально</p>
<p>-мы сами выбираем финишную вершину, когда задаем все остальные данные</p>
<p>герои ходят случайно, но 1) не ходят туда где уже были, если только они не в тупике; 2) если нашли финишную локацию, не уходят оттуда</p>

In [55]:
import random

In [56]:
land = {
    'A':['B', 'D'],
    'B' : ['A','C','D'],
    'C': ['B'],
    'D' :['A','B','E'],
    'E' : ['D']
}

goal = 'B'

In [57]:
heros = {
    'hero1' : ['A',['A']],
    'hero2' : ['E',['E']]
}

# 'имя героя': [вершина где он сейчас, [где он уже был]]
#первая вершина сразу содержится в списке посещенных

In [58]:
monsters = {
    'mon1' : 'C'
}

In [59]:
def check_heros():
    check = False
    for v in heros.values():
        if v[0] == goal:
            check = True
        else:
            check = False
            break
    return check
#v[0] -- это всегда текущая локация героя

In [60]:
#это функция, чтобы герой выбирал маршрут
#теперь надо ее вставить в ход героя

def find_path(land, visits, cur_node, goal):
    if goal == cur_node:
        return [cur_node]
    visits[cur_node] = True
    min_len = len(land) + 10
    l1 = None
    for loc in land[cur_node]:
        if visits[loc]:
            continue
        l = find_path(land, visits, loc, goal)
        if l != None and len(l) < min_len:
            l1 = [cur_node]
            l1.extend(l)
    return l1

#visited = visited.fromkeys(list(land.keys()), False)
#find_path(land, visited, 'B', 'C')


In [61]:
def update():
    go_monsters()
    go_heros1()
    
    if check_heros():
            print('Yay! Heros won!')
            return True
    else:
        for h,v in heros.items():
            if v[0] in monsters.values():
                print('Oh no! {} was caught :('.format(h))
                return True
    print('Continue to play')
    return

In [62]:
def go_monsters():
    for monster, loc in monsters.items():
        go_choice = random.randint(0,1)
        if go_choice == 1:
            loc_choice = random.choice(land[loc])
            monsters[monster] = loc_choice

In [63]:
#герои ходят случайно, но 1) не ходят туда где уже были, если только они не в тупике; 2) если нашли финишную локацию, не уходят оттуда
#c этим вариантом точно работает

def go_heros1():
    for hero, v in heros.items():
        if v[0] != goal:
            loc_choice = random.choice(land[v[0]])
            
            #ищем такую вершину, где он не был
            if len(land[v[0]])>1 and len(v[1])< len(land):
                while loc_choice in v[1]:
                    loc_choice = random.choice(land[v[0]])
                    
            #изменяем текущую
            heros[hero][0] = loc_choice
            
            #пополняем список пройденных вершин
            if heros[hero][0] not in heros[hero][1]:
                heros[hero][1].append(heros[hero][0])

In [64]:
#герои стараются идти по короткому пути
#так у меня не получилось доделать, чтобы работало

def go_heros2(visited = visited.fromkeys(list(land.keys()), False)):
    for hero, v in heros.items():
        if v[0] != goal:
            
            path = find_path(land, visited, v[0], goal)
            
            if len(path)>1:
                heros[hero][0] = path[1]
            else:
                heros[hero][0] = path[0]
            
            #пополняем список пройденных вершин
            if heros[hero][0] not in heros[hero][1]:
                heros[hero][1].append(heros[hero][0])

In [65]:
def print_the_land():
    print(heros, monsters, sep = '\n\n')

In [66]:
def play():
    #надо проверить, можно ли начинать игру
    if check_heros():
        print('The heros have already found the treasure!')
        return
    else:
        for h,v in heros.items():
            if v[0] in monsters.values():
                print('Sorry, but a monster has already caught {} :('.format(h))
                return
    
    #если нельзя, то игра не началась, если можно, то играем: 
    
    _ = ''
    quit = False
    while _ == '':
        _ = input()
        quit = update()
        print_the_land() #к какому раскладу привел этот ход
        if quit == True:
            break
            
#чтобы прошел один ход игры, нужно нажать Enter
#чтобы прервать игру, нужно что угодно ввести (не пусту строку)

In [67]:
print_the_land()

{'hero1': ['A', ['A']], 'hero2': ['E', ['E']]}

{'mon1': 'C'}


In [68]:
play()


Continue to play
{'hero1': ['D', ['A', 'D']], 'hero2': ['D', ['E', 'D']]}

{'mon1': 'C'}

Oh no! hero1 was caught :(
{'hero1': ['B', ['A', 'D', 'B']], 'hero2': ['A', ['E', 'D', 'A']]}

{'mon1': 'B'}
